In [1]:
#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

import gensim
from gensim import corpora
from pprint import pprint
from gensim.utils import simple_preprocess

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import pandas as pd

df = pd.read_csv("for_gensim.csv")

df.head()



,Unnamed: 0,Date,Time,formtype,Improve,Imp1,Imp2,ImpCrit,CommentCoderImp,TweetImp,Best,Best1,Best2,BestCrit,Division2,Directorate2,Location,Keep_Improve,comment,word_count
0,1,2009-04-01,1,adult,the time it took from the gps referral to gett...,NaN,NaN,NaN,NaN,NaN,X was an excellent therapist who helped me a lot.,NaN,NaN,NaN,Local partnerships- MH,NaN,Local partnerships- MH,The time it took from the G.P's referral to ge...,1,20
1,2,2009-04-01,1,adult,an advocate to support me when im ill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Local partnerships- MH,NaN,Local partnerships- MH,An advocate to support me when I'm ill,2,8
2,3,2009-04-01,1,adult,at this time i am unsure at to what could be i...,NaN,NaN,NaN,NaN,NaN,Meeting the team who are caring for my wife in...,NaN,NaN,NaN,Local partnerships- MH,NaN,Local partnerships- MH,At this time I am unsure at to what could be i...,3,12
3,4,2009-04-01,1,adult,at times the service can be a bit slow eg 3 we...,NaN,NaN,NaN,NaN,NaN,Felt very reassured and supported at all times...,NaN,NaN,NaN,Local partnerships- MH,NaN,Local partnerships- MH,"At times the service can be a bit slow, e.g. 3...",4,15
4,5,2009-04-01,1,adult,being told what is going on and helped to unde...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Local partnerships- MH,NaN,Local partnerships- MH,Being told what is going on and helped to unde...,5,13


In [2]:
# set up dataframe

df['Date'] = pd.to_datetime(df['Date'])

df["tokens"] = df["Improve"].apply(lambda x: x.split()).tolist()

print(df.head())


   Unnamed: 0       Date  Time formtype  \
0           1 2009-04-01     1    adult   
1           2 2009-04-01     1    adult   
2           3 2009-04-01     1    adult   
3           4 2009-04-01     1    adult   
4           5 2009-04-01     1    adult   

                                             Improve  Imp1  Imp2  ImpCrit  \
0  the time it took from the gps referral to gett...   NaN   NaN      NaN   
1              an advocate to support me when im ill   NaN   NaN      NaN   
2  at this time i am unsure at to what could be i...   NaN   NaN      NaN   
3  at times the service can be a bit slow eg 3 we...   NaN   NaN      NaN   
4  being told what is going on and helped to unde...   NaN   NaN      NaN   

  CommentCoderImp  TweetImp  \
0             NaN       NaN   
1             NaN       NaN   
2             NaN       NaN   
3             NaN       NaN   
4             NaN       NaN   

                         ...                         Best1  Best2  BestCrit  \
0           

In [3]:
%%time

# fit the model

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df["tokens"])]

# model = gensim.models.Doc2Vec(documents, dm = 0, alpha = 0.025, vector_size = 20, 
#                              min_alpha = 0.025, min_count = 0, epoch = 20)

# fit a model with a decay in the learning rate

model = Doc2Vec(alpha=0.025, min_alpha=0.025, vec_size = 100, workers = 4)  # use fixed learning rate
model.build_vocab(documents)
for epoch in range(20):
    model.train(documents, total_examples=model.corpus_count, epochs = 100)
    model.alpha -= 0.002  # decrease the learning rate
    model.min_alpha = model.alpha  # fix the learning rate, no decay
                              
print("No output here")                              


2019-02-15 10:16:21,870 : INFO : collecting all words and their counts
2019-02-15 10:16:21,874 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2019-02-15 10:16:21,961 : INFO : PROGRESS: at example #10000, processed 144611 words (1720672/s), 7777 word types, 10000 tags
2019-02-15 10:16:22,052 : INFO : PROGRESS: at example #20000, processed 296192 words (1725611/s), 11649 word types, 20000 tags
2019-02-15 10:16:22,126 : INFO : PROGRESS: at example #30000, processed 390716 words (1349653/s), 13645 word types, 30000 tags
2019-02-15 10:16:22,222 : INFO : collected 16588 word types and 39914 unique tags from a corpus of 39914 examples and 543285 words
2019-02-15 10:16:22,223 : INFO : Loading a fresh vocabulary
2019-02-15 10:16:22,251 : INFO : min_count=5 retains 4371 unique words (26% of original 16588, drops 12217)
2019-02-15 10:16:22,252 : INFO : min_count=5 leaves 525243 word corpus (96% of original 543285, drops 18042)
2019-02-15 10:16:22,288 : INFO : dele

2019-02-15 10:16:51,965 : INFO : EPOCH 9 - PROGRESS: at 59.16% examples, 124301 words/s, in_qsize 7, out_qsize 0
2019-02-15 10:16:53,009 : INFO : EPOCH 9 - PROGRESS: at 97.37% examples, 129654 words/s, in_qsize 3, out_qsize 1
2019-02-15 10:16:53,011 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-02-15 10:16:53,027 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-02-15 10:16:53,031 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-02-15 10:16:53,046 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-02-15 10:16:53,047 : INFO : EPOCH - 9 : training on 543285 raw words (427433 effective words) took 3.2s, 135056 effective words/s
2019-02-15 10:16:54,124 : INFO : EPOCH 10 - PROGRESS: at 27.57% examples, 116847 words/s, in_qsize 7, out_qsize 0
2019-02-15 10:16:55,128 : INFO : EPOCH 10 - PROGRESS: at 56.65% examples, 121267 words/s, in_qsize 7, out_qsize 0
2019-02-15 10:16:56,143 : INFO : EPOCH 10 - PRO

2019-02-15 10:17:23,078 : INFO : EPOCH 19 - PROGRESS: at 26.01% examples, 116113 words/s, in_qsize 7, out_qsize 0
2019-02-15 10:17:24,080 : INFO : EPOCH 19 - PROGRESS: at 59.16% examples, 129232 words/s, in_qsize 8, out_qsize 0
2019-02-15 10:17:25,120 : INFO : EPOCH 19 - PROGRESS: at 91.52% examples, 122289 words/s, in_qsize 7, out_qsize 0
2019-02-15 10:17:25,275 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-02-15 10:17:25,281 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-02-15 10:17:25,296 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-02-15 10:17:25,298 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-02-15 10:17:25,299 : INFO : EPOCH - 19 : training on 543285 raw words (427799 effective words) took 3.2s, 132751 effective words/s
2019-02-15 10:17:26,342 : INFO : EPOCH 20 - PROGRESS: at 33.30% examples, 150077 words/s, in_qsize 8, out_qsize 0
2019-02-15 10:17:27,362 : INFO : EPOCH 20 - 

2019-02-15 10:17:56,125 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-02-15 10:17:56,136 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-02-15 10:17:56,140 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-02-15 10:17:56,142 : INFO : EPOCH - 29 : training on 543285 raw words (427492 effective words) took 2.9s, 146877 effective words/s
2019-02-15 10:17:57,208 : INFO : EPOCH 30 - PROGRESS: at 28.84% examples, 125395 words/s, in_qsize 7, out_qsize 0
2019-02-15 10:17:58,253 : INFO : EPOCH 30 - PROGRESS: at 61.70% examples, 127238 words/s, in_qsize 7, out_qsize 0
2019-02-15 10:17:59,269 : INFO : EPOCH 30 - PROGRESS: at 97.03% examples, 129245 words/s, in_qsize 4, out_qsize 0
2019-02-15 10:17:59,317 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-02-15 10:17:59,320 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-02-15 10:17:59,332 : INFO : worker thread finished; awaiting f

2019-02-15 10:18:27,170 : INFO : EPOCH - 39 : training on 543285 raw words (427256 effective words) took 3.1s, 136110 effective words/s
2019-02-15 10:18:28,209 : INFO : EPOCH 40 - PROGRESS: at 27.57% examples, 120324 words/s, in_qsize 7, out_qsize 0
2019-02-15 10:18:29,249 : INFO : EPOCH 40 - PROGRESS: at 61.70% examples, 128873 words/s, in_qsize 7, out_qsize 0
2019-02-15 10:18:30,179 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-02-15 10:18:30,212 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-02-15 10:18:30,216 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-02-15 10:18:30,227 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-02-15 10:18:30,230 : INFO : EPOCH - 40 : training on 543285 raw words (427241 effective words) took 3.1s, 139970 effective words/s
2019-02-15 10:18:31,278 : INFO : EPOCH 41 - PROGRESS: at 28.84% examples, 126669 words/s, in_qsize 8, out_qsize 0
2019-02-15 10:18:32,31

...

2019-02-15 11:53:20,071 : INFO : EPOCH 64 - PROGRESS: at 30.30% examples, 138272 words/s, in_qsize 7, out_qsize 0
2019-02-15 11:53:21,089 : INFO : EPOCH 64 - PROGRESS: at 64.26% examples, 135844 words/s, in_qsize 7, out_qsize 0
2019-02-15 11:53:22,092 : INFO : EPOCH 64 - PROGRESS: at 97.37% examples, 133854 words/s, in_qsize 3, out_qsize 1
2019-02-15 11:53:22,093 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-02-15 11:53:22,113 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-02-15 11:53:22,116 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-02-15 11:53:22,122 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-02-15 11:53:22,123 : INFO : EPOCH - 64 : training on 543285 raw words (427478 effective words) took 3.1s, 139673 effective words/s
2019-02-15 11:53:23,158 : INFO : EPOCH 65 - PROGRESS: at 33.30% examples, 151703 words/s, in_qsize 7, out_qsize 0
2019-02-15 11:53:24,183 : INFO : EPOCH 65 - 

2019-02-15 11:53:52,584 : INFO : EPOCH 75 - PROGRESS: at 33.30% examples, 152158 words/s, in_qsize 7, out_qsize 0
2019-02-15 11:53:53,633 : INFO : EPOCH 75 - PROGRESS: at 66.99% examples, 137190 words/s, in_qsize 7, out_qsize 0
2019-02-15 11:53:54,391 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-02-15 11:53:54,412 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-02-15 11:53:54,416 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-02-15 11:53:54,418 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-02-15 11:53:54,419 : INFO : EPOCH - 75 : training on 543285 raw words (427562 effective words) took 2.9s, 149896 effective words/s
2019-02-15 11:53:55,470 : INFO : EPOCH 76 - PROGRESS: at 33.30% examples, 148371 words/s, in_qsize 7, out_qsize 0
2019-02-15 11:53:56,547 : INFO : EPOCH 76 - PROGRESS: at 66.99% examples, 133686 words/s, in_qsize 8, out_qsize 0
2019-02-15 11:53:57,270 : INFO : worker thre

2019-02-15 11:54:26,493 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-02-15 11:54:26,512 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-02-15 11:54:26,526 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-02-15 11:54:26,540 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-02-15 11:54:26,541 : INFO : EPOCH - 86 : training on 543285 raw words (427453 effective words) took 3.0s, 144764 effective words/s
2019-02-15 11:54:27,610 : INFO : EPOCH 87 - PROGRESS: at 33.30% examples, 147771 words/s, in_qsize 7, out_qsize 0
2019-02-15 11:54:28,679 : INFO : EPOCH 87 - PROGRESS: at 66.87% examples, 133817 words/s, in_qsize 7, out_qsize 0
2019-02-15 11:54:29,434 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-02-15 11:54:29,444 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-02-15 11:54:29,456 : INFO : worker thread finished; awaiting finish of 1 more threads

2019-02-15 11:54:58,798 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-02-15 11:54:58,799 : INFO : EPOCH - 97 : training on 543285 raw words (427576 effective words) took 3.0s, 143826 effective words/s
2019-02-15 11:54:59,828 : INFO : EPOCH 98 - PROGRESS: at 33.30% examples, 152310 words/s, in_qsize 7, out_qsize 0
2019-02-15 11:55:00,906 : INFO : EPOCH 98 - PROGRESS: at 66.87% examples, 135315 words/s, in_qsize 7, out_qsize 0
2019-02-15 11:55:01,684 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-02-15 11:55:01,696 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-02-15 11:55:01,703 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-02-15 11:55:01,721 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-02-15 11:55:01,722 : INFO : EPOCH - 98 : training on 543285 raw words (427320 effective words) took 2.9s, 146842 effective words/s
2019-02-15 11:55:02,807 : INFO : EPOCH 99 - P

No output here
CPU times: user 1h 36min 25s, sys: 23min 8s, total: 1h 59min 33s
Wall time: 1h 38min 45s


In [4]:
%%time

def similar_documents(comment):
    store_similar = []
    tokens = comment
    new_vector = model.infer_vector(tokens)
    sims = model.docvecs.most_similar([new_vector])
    res_list = [x[0] for x in sims]
    store_similar.append(res_list)
    return store_similar
    
df["similar_vector"] = df["Improve"].apply(lambda x: similar_documents(x))


2019-02-15 11:55:07,647 : INFO : precomputing L2-norms of doc weight vectors


CPU times: user 5min 40s, sys: 6min 37s, total: 12min 18s
Wall time: 3min 45s


In [5]:
# save

df.to_csv(path_or_buf = "/home/chris/Nextcloud/DataScienceAccelerator/shiny_doc2vec/document_results.csv", index = True)